In [1]:
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.utils import np_utils
from numpy.lib.npyio import load
from dense import Dense
from convolutional import Convolutional
from maxpool2d import Maxpool2d
from reshape import Reshape
from activations import Sigmoid, Tanh, Relu, Softmax
from losses import binary_cross_entropy, binary_cross_entropy_prime
from network import train

def preprocess_data(x, y, limit):
    # zero_index = np.where(y == 0)[0][:limit]
    # one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack(list(np.where(y == x)[0][:limit] for x in range(10)))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 10, 1)
    return x, y

# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_valid, y_valid = x_train[900:], y_train[900:]
x_train, y_train = x_train[:900], y_train[:900]
x_test, y_test = preprocess_data(x_test, y_test, 100)

l2 = 1e-1
learning_rate = 3e-2

# neural network
network = [
    Convolutional((1, 28, 28), 3, 10, l2=l2),
    Maxpool2d(0, 2, 2, (10, 13, 13)),
    Tanh(),
    Convolutional((10, 13, 13), 3, 10, l2=l2),
    Maxpool2d(1, 2, 2, (10, 6, 6)),
    Tanh(),
    Reshape((10, 6, 6), (10 * 6 * 6, 1)),
    Dense(10 * 6 * 6, 100, l2=l2),
    Tanh(),
    Dense(100, 10, l2=l2),
    Softmax()
]


Using TensorFlow backend.


In [3]:
[str(layer) for layer in network]

['Convolutional((1, 28, 28), 3, 10, l2=0.1)',
 'Maxpool2d(0, 2, 2, (10, 13, 13))',
 'Tanh',
 'Convolutional((10, 13, 13), 3, 10, l2=0.1)',
 'Maxpool2d(1, 2, 2, (10, 6, 6))',
 'Tanh',
 'Reshape((10, 6, 6), (360, 1))',
 'Dense(360, 100, l2=0.1)',
 'Tanh',
 'Dense(100, 10, l2=0.1)',
 'Softmax']